<a href="https://colab.research.google.com/github/rtegao/M5Forecasting/blob/master/ProductSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import plotly.express as px

In [0]:
df_sales = pd.read_csv('/content/drive/My Drive/Specialization/Projects/Recurrent Neural Networks/M5 - Data/sales_train_validation.csv')
df_calendar = pd.read_csv('/content/drive/My Drive/Specialization/Projects/Recurrent Neural Networks/M5 - Data/calendar.csv')
df_price = pd.read_csv('/content/drive/My Drive/Specialization/Projects/Recurrent Neural Networks/M5 - Data/calendar.csv')

In [20]:
df_calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [21]:
foods = df_sales[df_sales['cat_id']=='FOODS'].reset_index()
foods

,index,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,4,2,0,2,0,0,0,0,3,1,3,0,2,1,2,0,2,1,2,0,2,2,2,4,2,2,0,2,1,...,0,2,1,2,2,0,2,2,0,1,1,0,2,1,1,0,4,0,0,4,1,3,0,1,0,2,2,0,1,1,0,2,0,4,1,1,0,1,1,0
1,1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,1,1,0,0,0,0,1,1,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,...,1,1,2,1,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,3,1,0,0,1,2,0,0,0
2,1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,0,0,0,0,1,3,1,1,1,0,1,0,0,2,1,1,1,0,0,1,0,2,4,1,1,2,1,0,1,0,1,0,1,...,1,0,1,0,3,1,2,1,0,0,1,1,4,0,0,1,1,0,0,1,0,1,1,0,1,0,0,1,2,0,3,0,2,1,1,0,1,0,1,0
3,1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,3,9,3,3,0,2,1,2,1,7,4,0,2,3,6,15,5,0,0,0,1,1,2,3,0,3,4,1,3,1,4,0,2,...,2,1,1,1,3,2,3,0,0,0,2,3,1,1,0,0,3,0,0,2,0,0,3,0,2,0,2,0,1,2,3,1,1,2,0,2,2,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14365,30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,0,3,1,4,1,0,0,3,4,4,0,0,1,0,1,1,7,7,3,6,3,3,7,12,4,2,7,5,12,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,1,0,0,1
14366,30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,1,0,0,0,0,3,1,1,1,2,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0
14367,30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,4,7,2,3,5,6,3,4,1,2,5,1,2,2,...,1,1,2,0,0,2,0,0,0,2,0,1,1,0,0,0,0,1,1,2,1,4,0,0,2,2,4,0,0,0,2,1,0,2,0,1,0,0,1,0
14368,30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,2,1,1,2,4,3,0,0,0,0,0,1,1,2,1,1,1,1,2,0,1,0,3,0,0,1,0,0,1,0,3,1,3


In [22]:
np.isin(df_calendar['event_name_2'].fillna(0).unique(),df_calendar['event_name_1'].fillna(0).unique())

array([ True,  True,  True,  True,  True])

In [23]:
np.isin(df_calendar['event_type_2'].fillna(0).unique(),df_calendar['event_type_1'].fillna(0).unique())

array([ True,  True,  True])

In [0]:
#To categorial event name
numeric = list(range(len(df_calendar['event_name_1'].fillna(0).unique())))
categorical = df_calendar['event_name_1'].fillna(0).unique()
event_name = {categorical[i]:numeric[i] for i in range(len(categorical))}


#To categorical event type
numeric = list(range(len(df_calendar['event_type_1'].fillna(0).unique())))
categorical = df_calendar['event_type_1'].fillna(0).unique()
event_type = {categorical[i]:numeric[i] for i in range(len(categorical))}

In [0]:
corr = []
for i in range(len(foods)):
  snap = df_calendar.columns[df_calendar.columns.str.contains(foods.iloc[i,6])][0]
  food = foods.iloc[i,7:].to_frame(name='sales_qty').reset_index().set_index('index').join(df_calendar.set_index('d')).loc[:,['sales_qty','event_name_1','event_name_2','event_type_1','event_type_2',snap]].fillna(0).replace(event_name).copy()
  food = food.replace(event_type)
  corr.append(abs(food.corr().iloc[1:,0]).mean())

In [0]:
snap = df_calendar.columns[df_calendar.columns.str.contains(foods.iloc[corr.index(max(corr)),6])][0]
food = foods.iloc[corr.index(max(corr)),7:].to_frame(name='sales_qty').reset_index().set_index('index').join(df_calendar.set_index('d')).loc[:,['sales_qty','event_name_1','event_name_2','event_type_1','event_type_2',snap]].fillna(0).replace(event_name).copy()
food = food.replace(event_type)

In [66]:
food.corr()

,sales_qty,event_name_1,event_name_2,event_type_1,event_type_2,snap_WI
sales_qty,1.000000,-0.070969,0.019907,-0.050755,-0.007703,0.397771
event_name_1,-0.070969,1.000000,0.091951,0.831462,0.151020,-0.029160
event_name_2,0.019907,0.091951,1.000000,0.134109,0.747016,0.005767
event_type_1,-0.050755,0.831462,0.134109,1.000000,0.122162,-0.010899
event_type_2,-0.007703,0.151020,0.747016,0.122162,1.000000,0.006504
snap_WI,0.397771,-0.029160,0.005767,-0.010899,0.006504,1.000000


In [78]:
foods.iloc[corr.index(max(corr)),:]

index                            30160
id         FOODS_3_495_WI_3_validation
item_id                    FOODS_3_495
dept_id                        FOODS_3
cat_id                           FOODS
                      ...             
d_1909                               1
d_1910                               2
d_1911                               4
d_1912                               2
d_1913                               0
Name: 14040, Length: 1920, dtype: object

In [0]:
hobbies = df_sales[df_sales['cat_id']=='HOBBIES'].reset_index()
corr = []
for i in range(len(hobbies)):
  snap = df_calendar.columns[df_calendar.columns.str.contains(hobbies.iloc[i,6])][0]
  hobbie = hobbies.iloc[i,7:].to_frame(name='sales_qty').reset_index().set_index('index').join(df_calendar.set_index('d')).loc[:,['sales_qty','event_name_1','event_name_2','event_type_1','event_type_2',snap]].fillna(0).replace(event_name).copy()
  hobbie = hobbie.replace(event_type)
  corr.append(abs(hobbie.corr().iloc[1:,0]).mean())

In [80]:
snap = df_calendar.columns[df_calendar.columns.str.contains(hobbies.iloc[corr.index(max(corr)),6])][0]
hobbie = hobbies.iloc[corr.index(max(corr)),7:].to_frame(name='sales_qty').reset_index().set_index('index').join(df_calendar.set_index('d')).loc[:,['sales_qty','event_name_1','event_name_2','event_type_1','event_type_2',snap]].fillna(0).replace(event_name).copy()
hobbie = hobbie.replace(event_type)
hobbie.corr()

,sales_qty,event_name_1,event_name_2,event_type_1,event_type_2,snap_TX
sales_qty,1.000000,0.031132,0.092754,0.011398,0.252262,0.035024
event_name_1,0.031132,1.000000,0.091951,0.831462,0.151020,0.018733
event_name_2,0.092754,0.091951,1.000000,0.134109,0.747016,0.005767
event_type_1,0.011398,0.831462,0.134109,1.000000,0.122162,0.007689
event_type_2,0.252262,0.151020,0.747016,0.122162,1.000000,0.006504
snap_TX,0.035024,0.018733,0.005767,0.007689,0.006504,1.000000


In [81]:
hobbies.iloc[corr.index(max(corr)),:]

index                              12222
id         HOBBIES_1_027_TX_1_validation
item_id                    HOBBIES_1_027
dept_id                        HOBBIES_1
cat_id                           HOBBIES
                       ...              
d_1909                                 0
d_1910                                 0
d_1911                                 0
d_1912                                 0
d_1913                                 0
Name: 2286, Length: 1920, dtype: object

In [0]:
households = df_sales[df_sales['cat_id']=='HOUSEHOLD'].reset_index()
corr = []
for i in range(len(households)):
  snap = df_calendar.columns[df_calendar.columns.str.contains(households.iloc[i,6])][0]
  holsehold = households.iloc[i,7:].to_frame(name='sales_qty').reset_index().set_index('index').join(df_calendar.set_index('d')).loc[:,['sales_qty','event_name_1','event_name_2','event_type_1','event_type_2',snap]].fillna(0).replace(event_name).copy()
  holsehold = holsehold.replace(event_type)
  corr.append(abs(holsehold.corr().iloc[1:,0]).mean())

In [83]:
snap = df_calendar.columns[df_calendar.columns.str.contains(households.iloc[corr.index(max(corr)),6])][0]
household = households.iloc[corr.index(max(corr)),7:].to_frame(name='sales_qty').reset_index().set_index('index').join(df_calendar.set_index('d')).loc[:,['sales_qty','event_name_1','event_name_2','event_type_1','event_type_2',snap]].fillna(0).replace(event_name).copy()
household = household.replace(event_type)
household.corr()

,sales_qty,event_name_1,event_name_2,event_type_1,event_type_2,snap_TX
sales_qty,1.000000,0.028597,0.365170,0.074787,0.170719,0.041841
event_name_1,0.028597,1.000000,0.091951,0.831462,0.151020,0.018733
event_name_2,0.365170,0.091951,1.000000,0.134109,0.747016,0.005767
event_type_1,0.074787,0.831462,0.134109,1.000000,0.122162,0.007689
event_type_2,0.170719,0.151020,0.747016,0.122162,1.000000,0.006504
snap_TX,0.041841,0.018733,0.005767,0.007689,0.006504,1.000000


In [84]:
households.iloc[corr.index(max(corr)),:]

index                                16852
id         HOUSEHOLD_2_512_TX_2_validation
item_id                    HOUSEHOLD_2_512
dept_id                        HOUSEHOLD_2
cat_id                           HOUSEHOLD
                        ...               
d_1909                                   0
d_1910                                   1
d_1911                                   1
d_1912                                   0
d_1913                                   3
Name: 6277, Length: 1920, dtype: object